In [1]:
import gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete
import numpy as np
import random
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

# Types of Spaces

In [2]:
Discrete(3)

Discrete(3)

In [4]:
Box(0,1,shape=(3,)).sample()

array([0.3510414 , 0.7454274 , 0.15342325], dtype=float32)

In [7]:
Tuple((Discrete(3), Box(0,1, shape=(3,)))).sample()

(2, array([0.19887662, 0.57330024, 0.40109503], dtype=float32))

In [8]:
Dict({'height':Discrete(2), "speed":Box(0,100, shape=(1,))}).sample()

OrderedDict([('height', 1), ('speed', array([47.622215], dtype=float32))])

In [12]:
MultiBinary(4).sample()

array([0, 0, 1, 1], dtype=int8)

In [13]:
MultiDiscrete([5,2,2]).sample()

array([4, 1, 0], dtype=int64)

# Build Environment

In [14]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60

    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0
        # 2 -1 = 1 temperature
        self.state += action -1
        # Reduce shower length by 1 second
        self.shower_length -= 1

        # Calculate reward
        if self.state >=37 and self.state <=39:
            reward =1
        else:
            reward = -1

        # Check if shower is done
        if self.shower_length <= 0:
            done = True
        else:
            done = False

        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}

        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass

    def reset(self):
        # Reset shower temperature
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        # Reset shower time
        self.shower_length = 60
        return self.state

In [15]:
env=ShowerEnv()

C:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\gym\spaces\box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [17]:
env.observation_space.sample()

array([9.611827], dtype=float32)

In [18]:
env.reset()

array([39.])

# Test

In [19]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:-4
Episode:2 Score:32
Episode:3 Score:-12
Episode:4 Score:-44
Episode:5 Score:-2


# Training

In [20]:
log_path = os.path.join('Training', 'Logs')

In [21]:
log_path

'Training\\Logs'

In [22]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [23]:
model.learn(total_timesteps=100000)

Logging to Training\Logs\PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -34.6    |
| time/              |          |
|    fps             | 1913     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60           |
|    ep_rew_mean          | -35.3        |
| time/                   |              |
|    fps                  | 1508         |
|    iterations           | 2            |
|    time_elapsed         | 2            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0038898843 |
|    clip_fraction        | 0.00659      |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.09        |
|    explained_variance   |

# Save

In [24]:
save_path = os.path.join('Training', 'Saved_Model', 'Temp_PPO')

In [25]:
save_path

'Training\\Saved_Model\\Temp_PPO'

In [26]:
model.save(save_path)

C:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\stable_baselines3\common\save_util.py:278: UserWarning: Path 'Training\Saved_Model' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [27]:
del model

In [28]:
model = PPO.load(save_path, env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [29]:
evaluate_policy(model, env, n_eval_episodes=10)

C:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(59.4, 0.9165151389911681)